<img width="8%" alt="HubSpot.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/HubSpot.png" style="border-radius: 15%">

# HubSpot - Create task
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=HubSpot+-+Create+task:+Error+short+description">Bug report</a>

**Tags:** #hubspot #sales #crm #engagements #task #snippet

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-11-21 (Created: 2023-11-21)

**Description:** This notebook demonstrates how to create a task in HubSpot associated with contacts, companies or deals.

**References:**
- [HubSpot API - Tasks](https://developers.hubspot.com/docs/api/crm/tasks)
- [Get your HubSpot Access token](https://knowledge.hubspot.com/articles/kcs_article/integrations/how-do-i-get-my-hubspot-api-key)

## Input

### Import libraries

In [ ]:
import requests
import naas
from datetime import datetime, date, timezone

### Setup variables
- `hs_access_token`: This variable stores an access token used for accessing the HubSpot API.
- `hs_task_subject`: The title of the task.
- `hs_task_type`: The type of task. Values include EMAIL, CALL, or TODO.
- `hs_task_body`: The task notes.
- `hs_task_priority`: The priority of the task. Values include LOW, MEDIUM, or HIGH.
- `hs_task_status`: The status of the task, either NOT_STARTED, COMPLETED, IN_PROGRESS, WAITING, DEFERRED
- `hs_timestamp`: Required. This field marks the task's due date. You can use either a Unix timestamp in milliseconds or UTC format.
- `hubspot_owner_id`: The owner ID of the user assigned to the task.
- `contact_associations`: The list of contact IDs associated to the task.
- `company_associations`: The list of company IDs associated to the task.
- `deal_associations`: The list of deal IDs associated to the task.

In [ ]:
hs_access_token = naas.secret.get("HS_ACCESS_TOKEN") or "YOUR_HS_ACCESS_TOKEN"
hs_task_subject = "My new task"
hs_task_type = "TODO"
hs_task_body = "Perform the following actions: ...."
hs_task_priority = "HIGH"
hs_task_status = "NOT_STARTED"
hs_timestamp = None
hubspot_owner_id = None
contact_associations = ["693551"]
company_associations = ["16009427212"]
deal_associations = ["15020814750"]

## Model

### Create task

In [ ]:
def create_task(
    hs_access_token,
    hs_task_subject,
    hs_task_type="TODO",
    hs_task_body=None,
    hs_task_priority=None,
    hs_task_status=None,
    hs_timestamp=None,
    hubspot_owner_id=None,
    contact_ids=[],
    company_ids=[],
    deal_ids=[]
):
    # Init
    data = []
    associations = []
    
    # Get the current timestamp in UTC
    if not hs_timestamp:
        hs_timestamp = datetime.utcnow().replace(tzinfo=timezone.utc).strftime("%s") + "000"
        
    # Create contact associations
    contacts = []
    for object_id in contact_ids:
        contacts.append(
            {
                "to": {"id": object_id},
                "types": [
                    {
                        "associationCategory": "HUBSPOT_DEFINED",
                        "associationTypeId": 204
                    }
                ]
            }
        )
        
    # Create contact asso
    companies = []
    for object_id in company_ids:
        companies.append(
            {
                "to": {"id": object_id},
                "types": [
                    {
                        "associationCategory": "HUBSPOT_DEFINED",
                        "associationTypeId": 192
                    }
                ]
            }
        )
        
    # Create contact asso
    deals = []
    for object_id in deal_ids:
        deals.append(
            {
                "to": {"id": object_id},
                "types": [
                    {
                        "associationCategory": "HUBSPOT_DEFINED",
                        "associationTypeId": 216
                    }
                ]
            }
        )
        
    # Requests
    associations = contacts + companies + deals
    payload = {
        "properties":
        {
            "hs_task_subject": hs_task_subject,
            "hs_task_type": hs_task_type,
            "hs_task_body": hs_task_body,
            "hs_task_status": hs_task_status,
            "hs_task_priority": hs_task_priority,
            "hs_timestamp": hs_timestamp,
            "hubspot_owner_id": hubspot_owner_id,
        },
        "associations": associations
    }
    headers = {
        'accept': "application/json",
        'content-type': "application/json",
        'authorization': f"Bearer {hs_access_token}"
    }
    url = "https://api.hubapi.com/crm/v3/objects/tasks"
    
    # Response
    res = requests.post(url, headers=headers, json=payload)
    if res.status_code == 201:
        data = res.json()
        print("✅ Task successfully created:", data.get('id'))
    else:
        print(res)
        print(res.text)
    return data

task = create_task(
    hs_access_token,
    hs_task_subject,
    hs_task_type=hs_task_type,
    hs_task_body=hs_task_body,
    hs_task_priority=hs_task_priority,
    hs_task_status=hs_task_status,
    hs_timestamp=hs_timestamp,
    hubspot_owner_id=hubspot_owner_id,
    contact_ids=contact_associations,
    company_ids=company_associations,
    deal_ids=deal_associations,
)

## Output

### Display result

In [ ]:
task